In [ ]:
# Run packages

import numpy as np
import cv2
import matplotlib.pyplot as plt
import PyDroneCurrents as pdc
import sys
sys.path.append('c:\Flavia\Mestrado\Traducao_codigo')  #Put the path that contain the 'PyDroneCurrents'


## Create georeference structure

In [ ]:
#Path where video is localized:
video_name = r'F:\Flavia_drone\Ponte_franceses_20230607\Primeira_tomada\100_metros\DJI_0011.MP4'

# Add metadata in dictionary form:
# To obtain the video's metadata, it is necessary to use a separate tool, such as MediaInfo.
CamPos_ST= {'LONGITUDE':- 52.088902,'LATITUDE': -32.058870,'Height': 100.7,'yaw': 18.3,'pitch': -89.9, 'roll': 0}

#Time in seconds between frames:
#Interval in seconds between extracted frames. Must be a multiple of the video frame rate. 
#If the selected dt is not a multiple of the video frame the value will be adjusted to the nearest multiple.
dt = 0.0834167

#Time of the video to be analyzed in seconds [initial_time, end_time] :
time_limits = [5,15]

#Distance between the home point and the water surface in meters:
offset_homewater_Z = 0.1

#Input of manually entered camera calibration data:
#The parameters used were obtained from the Phantom 4 pro V2 camera calibration, in 4K resolution.(3840X2160)
file_cal={'fov_diag':77.546012315234050, 'fov_x':69.990877420892020, 'fov_y':42.989041303399810,  'size_X': 3840, 'size_Y':2160, 'camera_offset_Z':0}

# file_cal:
    # fov_diag = diagonal field of view angle;
    # fov_x = x (horizontal) field of view angle;
    # fov_y = y (vertical) field of view angle;
    # size_X = frame pixel size in x (horizontal);
    # size_Y = frame pixel size in y (vertical);
    # camera_offset_Z = vertical (Z) distance in meters; 


In [ ]:
# Adjust the georeference structure
video_name, dt, offset_homewater_Z, file_cal = pdc.create_struct_georeference(video_name, dt, offset_homewater_Z, file_cal)

## Generate sequence image

In [ ]:
IMG_SEQ = pdc.run_georeference_struct(CamPos_ST, video_name, time_limits, file_cal, dt)

## Get Windows

In [ ]:
STCFIT = pdc.get_STCFIT_from_IMG_SEQ(IMG_SEQ)

## Run Currents

In [ ]:
FG_Ux, FG_Uy, SG_Ux, SG_Uy, STCFIT = pdc.run_currents(STCFIT, IMG_SEQ)

## Save velocity vector

#### If you can save the list of vectors velocity in a file txt for open separate, run the cell bellow:


In [ ]:
#Save the u vector (velocity in x):
np.savetxt('SG_Ux.csv', SG_Ux, delimiter=',',fmt='%f') # The file will save on the path that stay this nottebook run
# The file will save on the path that stay this notebook run, with the name SG_Ux

#Save the v vector (velocity in y)
np.savetxt('SG_Uy.csv', SG_Uy, delimiter=',',fmt='%f')
# The file will save on the path that stay this notebook run, with the name SG_Uy


## Plot currents in camera grid

In [ ]:
plot_vectors, x1, y1 = pdc.plot_currents_camera(SG_Ux, SG_Uy, STCFIT, CamPos_ST)

## Georeferenced vectors

In [ ]:
Ux_geo, Uy_geo, grid_long_geo, grid_lat_geo, long_geo, lat_geo= pdc.georeference_vectors(CamPos_ST,STCFIT, x1, y1, SG_Ux, SG_Uy)

## Plot georeference vectors

In [ ]:
vec_geo= pdc.plot_georeference_vectors(STCFIT, grid_long_geo, grid_lat_geo, long_geo, lat_geo, Ux_geo, Uy_geo)

## Plot windows

In [ ]:
plot_windows = pdc.plot_windows(STCFIT)